## Deliverable 2. Create a Customer Travel Destinations Map.

In [27]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import matplotlib.pyplot as plt

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [5]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Peterhead,GB,57.5058,-1.7981,46.71,76,100,28.72,overcast clouds
1,1,Punta Arenas,CL,-53.1500,-70.9167,51.91,46,40,28.77,scattered clouds
2,2,Arraial Do Cabo,BR,-22.9661,-42.0278,71.56,60,75,8.05,broken clouds
3,3,Chara,RU,56.9064,118.2631,30.51,76,100,2.48,overcast clouds
4,4,Plymouth,US,41.8334,-70.8328,77.22,62,1,1.01,clear sky


In [8]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

max_temp = float(input("What is the maximum temperature you'd like for your trip? "))
min_temp = float(input("What is the minimum temperature you'd like for your trip?"))

In [9]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Punta Arenas,CL,-53.1500,-70.9167,51.91,46,40,28.77,scattered clouds
10,10,Mar Del Plata,AR,-38.0023,-57.5575,61.88,90,19,17.00,few clouds
16,16,Chipinge,ZW,-20.1883,32.6236,62.20,68,64,7.45,broken clouds
22,22,Ahipara,NZ,-35.1667,173.1667,60.31,94,100,10.02,overcast clouds
24,24,Hobart,AU,-42.8794,147.3294,58.86,82,75,9.22,broken clouds
25,25,Challans,FR,46.8436,-1.8749,57.40,78,24,9.44,few clouds
27,27,Christchurch,NZ,-43.5333,172.6333,52.00,84,0,3.44,clear sky
38,38,Cape Town,ZA,-33.9258,18.4232,64.09,59,100,10.00,overcast clouds
42,42,Busselton,AU,-33.6500,115.3333,50.09,84,40,6.31,scattered clouds
49,49,Luderitz,NaN,-26.6481,15.1594,58.53,79,63,11.65,broken clouds


In [13]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                134
City                   134
Country                130
Lat                    134
Lng                    134
Max Temp               134
Humidity               134
Cloudiness             134
Wind Speed             134
Current Description    134
dtype: int64

In [17]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

clean_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Punta Arenas,CL,-53.1500,-70.9167,51.91,46,40,28.77,scattered clouds
10,10,Mar Del Plata,AR,-38.0023,-57.5575,61.88,90,19,17.00,few clouds
16,16,Chipinge,ZW,-20.1883,32.6236,62.20,68,64,7.45,broken clouds
22,22,Ahipara,NZ,-35.1667,173.1667,60.31,94,100,10.02,overcast clouds
24,24,Hobart,AU,-42.8794,147.3294,58.86,82,75,9.22,broken clouds
25,25,Challans,FR,46.8436,-1.8749,57.40,78,24,9.44,few clouds
27,27,Christchurch,NZ,-43.5333,172.6333,52.00,84,0,3.44,clear sky
38,38,Cape Town,ZA,-33.9258,18.4232,64.09,59,100,10.00,overcast clouds
42,42,Busselton,AU,-33.6500,115.3333,50.09,84,40,6.31,scattered clouds
50,50,Yichang,CN,30.7144,111.2847,57.51,76,100,1.54,light rain


In [18]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Punta Arenas,CL,51.91,scattered clouds,-53.1500,-70.9167,
10,Mar Del Plata,AR,61.88,few clouds,-38.0023,-57.5575,
16,Chipinge,ZW,62.20,broken clouds,-20.1883,32.6236,
22,Ahipara,NZ,60.31,overcast clouds,-35.1667,173.1667,
24,Hobart,AU,58.86,broken clouds,-42.8794,147.3294,
25,Challans,FR,57.40,few clouds,46.8436,-1.8749,
27,Christchurch,NZ,52.00,clear sky,-43.5333,172.6333,
38,Cape Town,ZA,64.09,overcast clouds,-33.9258,18.4232,
42,Busselton,AU,50.09,scattered clouds,-33.6500,115.3333,
50,Yichang,CN,57.51,light rain,30.7144,111.2847,


In [19]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# Iterate through the DataFrame
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found ... skipping.")
    
        

Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.
Hotel not found ... skipping.


In [21]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Punta Arenas,CL,51.91,scattered clouds,-53.1500,-70.9167,Hotel Dreams Del Estrecho
10,Mar Del Plata,AR,61.88,few clouds,-38.0023,-57.5575,Gran Hotel Mar del Plata
16,Chipinge,ZW,62.20,broken clouds,-20.1883,32.6236,Chipinge Breeze Lodge
22,Ahipara,NZ,60.31,overcast clouds,-35.1667,173.1667,Ahipara Holiday Park
24,Hobart,AU,58.86,broken clouds,-42.8794,147.3294,Mantra on Collins Hobart
25,Challans,FR,57.40,few clouds,46.8436,-1.8749,Citotel Le Challans
27,Christchurch,NZ,52.00,clear sky,-43.5333,172.6333,The George
38,Cape Town,ZA,64.09,overcast clouds,-33.9258,18.4232,Southern Sun Waterfront Cape Town
42,Busselton,AU,50.09,scattered clouds,-33.6500,115.3333,Observatory Guest House
50,Yichang,CN,57.51,light rain,30.7144,111.2847,Gezhouba Hotel


In [22]:
# 8a. Create the output File (CSV)
# Create the output file (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
## 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [23]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [29]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat","Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2.75)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>